In [1]:
import openai
import pandas as pd

In [2]:
with open('../papers/01.txt', 'r') as fh:
    tmp = fh.read()
    itemlist = tmp.split(',')

In [3]:
itemlist = str(itemlist)

In [4]:
keyfile = open("key.txt", "r")
key = keyfile.readline()
openai.api_key = key

In [5]:
data = [itemlist[0:3200], itemlist[3201:6400], 
        itemlist[6401:9600], itemlist[9601:12800]]

In [6]:
def chatgptfn(sub_list):
    result = ''
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant"},
            {"role": "user", "content": f"{sub_list} :give me a summary"}
        ]
    )
    for choice in response.choices:
        result += choice.message.content
    return result

In [7]:
data[0] = chatgptfn(data[0])

In [8]:
data[1] = chatgptfn(data[1])

In [9]:
data[2] = chatgptfn(data[2])
data[3] = chatgptfn(data[3])

In [10]:
import gensim
from gensim import corpora
from pprint import pprint

In [11]:
# 創建詞袋
texts = [[word for word in document.lower().split()] for document in data]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [12]:
# 訓練 LDA 模型
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, 
                                            num_topics=3, random_state=100, update_every=1, 
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [13]:
# 輸出主題模型分析結果
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\pecu6\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
